In [ ]:
import numpy as np
import pylab as pl
from pykalman import KalmanFilter

# specify parameters
random_state = np.random.RandomState(0)


In [ ]:
t = np.linspace(0, 2 * np.pi, 50)
t

In [ ]:
x = np.sin(t)
x

In [ ]:
x_noised = x + 0.1 *random_state.randn(*x.shape)
x_noised

In [ ]:
# Fixed description of vars
# Renamed vars in correct()
class KF:
    def __init__(self, q, r, f = 1, h = 1):
        self.F = 0.0 # factor of real value to previous real value
        self.Q = 0.0 # covariance of untracked influences
        self.H = 0.0 # factor of measured value to real value
        self.R = 0.0 # covariance of measurement noise
        self.Xprev = 0.0 # previous state
        self.Pprev = 0.0 # previous covariance

        self.Q = q
        self.R = r
        self.F = f
        self.H = h
    def set(self, x, covariance):
        self.Xprev = x
        self.Pprev = covariance
    def correct(self, data):
        # time update - prediction
        Xpredicted = self.F * self.Xprev
        Ppredicted = self.F * self.Pprev * self.F + self.Q

        # measurement update - correction
        K = self.H * Ppredicted / (self.H * Ppredicted * self.H + self.R)
        
        Xnew = Xpredicted + K * (data - self.H * Xpredicted)
        Pnew = Ppredicted - K * self.H * Ppredicted
        
        self.Xprev = Xnew
        self.Pprev = Pnew

def smooth(arr):
    out = []
    kf = KF(q=2, r=15, f=1, h=1)
    #kf = KF(q=1, r=10, f=1, h=1)
    kf.set(arr[0], 0)
    for d in arr:
        kf.correct(d)
        out.append(kf.Xprev)
    return out

In [ ]:
def filterPy(arr):
    kf = KalmanFilter(
        transition_matrices = np.eye(1), # F
        transition_covariance = [2], #np.eye(1), #Q
        transition_offsets = np.zeros(1),
        observation_matrices = np.eye(1), # H
        observation_covariance = [15], #np.eye(1), # R
        observation_offsets = np.zeros(1),
        initial_state_mean = [arr[0]],
        initial_state_covariance = [0]
    )
    return kf.filter(arr)[0]

def filter2Py(arr):
    kf = KalmanFilter(
        transition_matrices = np.eye(1), # F
        transition_covariance = [2], #np.eye(1), #Q
        transition_offsets = np.zeros(1),
        observation_matrices = np.eye(1), # H
        observation_covariance = [15], #np.eye(1), # R
        observation_offsets = np.zeros(1),
        initial_state_mean = [arr[0]],
        initial_state_covariance = [0]
    )
    mean, covariance = arr[0], 0
    out = []
    for d in arr:
        mean, covariance = kf.filter_update(mean, covariance, observation = d)
        out.append(mean.item(0))
    return out


def smoothPy(arr):
    kf = KalmanFilter(
        transition_matrices = np.eye(1), # F
        transition_covariance = [2], #np.eye(1), #Q
        transition_offsets = np.zeros(1),
        observation_matrices = np.eye(1), # H
        observation_covariance = [15], #np.eye(1), # R
        observation_offsets = np.zeros(1),
        initial_state_mean = [arr[0]],
        initial_state_covariance = [0]
    )
    return kf.smooth(arr)[0]

In [ ]:
pl.rcParams['figure.figsize'] = [10, 10]
pl.figure()
pl.plot(x, 'b-')
pl.plot(x_noised, 'g-')
pl.plot(smooth(x_noised), 'r-')
pl.plot(filterPy(x_noised), 'cx')
pl.plot(filter2Py(x_noised), 'co')
pl.plot(smoothPy(x_noised), 'rx')
pl.show()